In [ ]:
import numpy as np, pandas as pd, pickle, matplotlib.pyplot as plt

In [ ]:
datadir = '../../../datasets/ppmi/visit_feature_inputs_asof_2019Jan24_using_CMEDTM/'
pd_totals_df = pd.read_csv(datadir + 'PD_totals_across_time.csv')
outcome_path = '../ppmi_survival_models/survival_outcome_subtotals_gdsfixed_using_CMEDTM/set_3.0_0.5_2019Jul08/' \
    + 'cohorts_time_event_dict.pkl'
with open(outcome_path, 'r') as f:
    pd_surv_df = pickle.load(f)['PD']

### Determine a T for censoring approach

In [ ]:
last_visits_df = pd_totals_df.sort_values(by=['EVENT_ID_DUR']).drop_duplicates(subset=['PATNO'], keep='last')
last_visits_df.EVENT_ID_DUR.plot.hist()
plt.xlabel('Time enrolled (years)')
plt.ylabel('# patients')
plt.show()

In [ ]:
for i in range(10):
    print('# patients with at least ' + str(i) + ' years enrolled: ' \
          + str(len(last_visits_df.loc[last_visits_df['EVENT_ID_DUR']>=i])))

In [ ]:
pd_totals_df.columns

In [ ]:
pd_totals_df['NUPDRS3_any'] = np.where(~pd.isnull(pd_totals_df['NUPDRS3_untreated']), \
                                       pd_totals_df['NUPDRS3_untreated'], \
                                       np.where(~pd.isnull(pd_totals_df['NUPDRS3_off']), \
                                                pd_totals_df['NUPDRS3_off'], \
                                                np.where(~pd.isnull(pd_totals_df['NUPDRS3_on']), \
                                                         pd_totals_df['NUPDRS3_on'], \
                                                         pd_totals_df['NUPDRS3_maob'])))

In [ ]:
totals_used = ['SCOPA-AUT', 'HVLT_discrim_recog', 'HVLT_immed_recall', 'NUPDRS3_any', 'QUIP', 'EPWORTH', \
               'STATE_ANXIETY', 'TRAIT_ANXIETY', 'NUPDRS2', 'HVLT_retent', 'BJLO', 'MOCA', 'LNS', 'SEMANTIC_FLUENCY', \
               'REMSLEEP']

In [ ]:
# actually need 2nd to last visit for enroll time due to censoring
pd_totals_df['PATNO_EVENT_ID_DUR'] = pd_totals_df['PATNO'].astype(str) + pd_totals_df['EVENT_ID_DUR'].astype(str)
pd_totals_df_nonan = pd_totals_df.dropna(subset=totals_used)
last_visits_df = pd_totals_df_nonan.sort_values(by=['EVENT_ID_DUR']).drop_duplicates(subset=['PATNO'], keep='last')
second_to_last_visits_df \
    = pd_totals_df_nonan.loc[~pd_totals_df_nonan['PATNO_EVENT_ID_DUR'].isin(last_visits_df['PATNO_EVENT_ID_DUR'])]
second_to_last_visits_df \
    = second_to_last_visits_df.sort_values(by=['EVENT_ID_DUR']).drop_duplicates(subset=['PATNO'], keep='last')
second_to_last_visits_df.EVENT_ID_DUR.plot.hist()
plt.xlabel('Time enrolled (years)')
plt.ylabel('# patients')
plt.show()

In [ ]:
for i in range(10):
    print('# patients with at least ' + str(i) + ' years enrolled: ' \
          + str(len(second_to_last_visits_df.loc[second_to_last_visits_df['EVENT_ID_DUR']>=i])))

In [ ]:
pd_totals_df.EVENT_ID_DUR.unique()

In [ ]:
outcomes = ['Motor', 'Autonomic', 'Cognitive', 'Sleep', 'Psychiatric']
ts = [3.125, 3.625, 4.125, 4.625, 5.125]
fig, ax = plt.subplots(nrows=len(outcomes), ncols=len(ts), figsize=(12,10), sharex=True, sharey=True)
plt.rcParams.update({'font.size':12})
obs_before_ts_dict = dict()
obs_after_ts_dict = dict()
cens_after_ts_dict = dict()
for idx in range(len(ts)):
    obs_before_ts_dict[ts[idx]] = []
    obs_after_ts_dict[ts[idx]] = []
    cens_after_ts_dict[ts[idx]] = []
    patnos_enrolled_at_least_i_years \
        = set(second_to_last_visits_df.loc[second_to_last_visits_df['EVENT_ID_DUR']>=ts[idx]].PATNO.values.tolist())
    #print('Require at least ' + str(ts[idx]) + ' years of enrollment: ' \
    #      + str(len(patnos_enrolled_at_least_i_years)) + ' patients')
    pd_surv_df_atleast_i = pd_surv_df.loc[pd_surv_df['PATNO'].isin(patnos_enrolled_at_least_i_years)]
    for outcome_idx in range(len(outcomes)):
        outcome = outcomes[outcome_idx]
        assert len(pd_surv_df_atleast_i.loc[np.logical_and(pd_surv_df_atleast_i[outcome + '_T'] < ts[idx], \
                                                           pd_surv_df_atleast_i[outcome + '_E']==0)])== 0
        pd_surv_df_atleast_i[outcome + '_T_atleast' + str(ts[idx])] \
            = np.where(pd_surv_df_atleast_i[outcome + '_E']==0, ts[idx] + 0.25, \
                       pd_surv_df_atleast_i[outcome + '_T']) # set censored to max time
        pd_surv_df_atleast_i[outcome + '_T_atleast' + str(ts[idx])] \
            = np.where(pd_surv_df_atleast_i[outcome + '_T_atleast' + str(ts[idx])] >= ts[idx] + 0.25, ts[idx] + 0.25, \
                       pd_surv_df_atleast_i[outcome + '_T_atleast' + str(ts[idx])]) # set larger observed to max time
        ax[outcome_idx, idx].hist(pd_surv_df_atleast_i[outcome + '_T_atleast' + str(ts[idx])])
        ax[outcome_idx, idx].set_xlabel(outcome + ' (T=' + str(ts[idx]) + ')')
        ax[outcome_idx, idx].set_ylabel('# patients')
        #print(outcome + ': ' + str(pd_surv_df_atleast_i[outcome + '_E'].sum()) + ' observed, ' \
        #      + str(len(pd_surv_df_atleast_i) - pd_surv_df_atleast_i[outcome + '_E'].sum()) + ' censored')
        num_obs_before_t \
            = len(pd_surv_df_atleast_i.loc[np.logical_and(pd_surv_df_atleast_i[outcome + '_E']==1, \
                                                          pd_surv_df_atleast_i[outcome + '_T'] <= ts[idx] + 0.25)])
        obs_before_ts_dict[ts[idx]].append(num_obs_before_t)
        num_obs_after_t \
            = len(pd_surv_df_atleast_i.loc[np.logical_and(pd_surv_df_atleast_i[outcome + '_E']==1, \
                                                          pd_surv_df_atleast_i[outcome + '_T'] > ts[idx] + 0.25)])
        obs_after_ts_dict[ts[idx]].append(num_obs_after_t)
        num_cens_after_t = len(pd_surv_df_atleast_i.loc[pd_surv_df_atleast_i[outcome + '_E']==0])
        cens_after_ts_dict[ts[idx]].append(num_cens_after_t)
plt.suptitle('Imputed time to observation (yrs)')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('impute_obs_time_distribs.pdf')
plt.show()
if len(ts) == 3:
    x_offsets = [-0.1, 0.0, 0.1]
elif len(ts) == 4:
    x_offsets = [-0.15, -0.05, 0.05, 0.15]
elif len(ts) == 5:
    x_offsets = [-0.2, -0.1, 0, 0.1, 0.2]
fig, ax = plt.subplots(figsize=(8,6))
for idx in range(len(ts)):
    if idx == 0:
        labels = ['observed before T', 'observed after T', 'censored after T']
    else:
        labels = [None, None, None]
    ax.bar(np.arange(len(outcomes))+x_offsets[idx], obs_before_ts_dict[ts[idx]], align='center', width=0.09, \
           label=labels[0], color='red')
    ax.bar(np.arange(len(outcomes))+x_offsets[idx], obs_after_ts_dict[ts[idx]], align='center', width=0.09, \
           bottom=obs_before_ts_dict[ts[idx]], label=labels[1], color='green')
    ax.bar(np.arange(len(outcomes))+x_offsets[idx], cens_after_ts_dict[ts[idx]], align='center', width=0.09, \
           bottom=np.array(obs_before_ts_dict[ts[idx]]) + np.array(obs_after_ts_dict[ts[idx]]), \
           label=labels[2], color='blue')
ax.set_xticklabels([''] + outcomes)
ts_str = 'T = ('
for t in ts:
    ts_str += str(t) + ', '
ts_str = ts_str[:-2] + ')'
ax.set_xlabel(ts_str)
ax.set_ylabel('# patients')
plt.legend()
plt.savefig('obs_cens_around_T.pdf')
plt.show()

Setting T = 4.125 seems to be the best trade-off between sample size and cutting off too many observations. Psychiatric would be the only outcome that has a lot of observations after the cutoff.

### Visualize outcomes in pairwise plots and t-SNE

In [ ]:
from seaborn import heatmap
from sklearn.manifold import TSNE

In [ ]:
outcomes = ['Motor', 'Autonomic', 'Cognitive', 'Sleep', 'Psychiatric']
patnos_enrolled_at_least_i_years \
    = set(second_to_last_visits_df.loc[second_to_last_visits_df['EVENT_ID_DUR']>=4.125].PATNO.values.tolist())
pd_surv_df_atleast_i = pd_surv_df.loc[pd_surv_df['PATNO'].isin(patnos_enrolled_at_least_i_years)]
outcome_T_atleast_i_cols = []
for outcome in outcomes:
    pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] \
        = np.where(pd_surv_df_atleast_i[outcome + '_E']==0, 4.125 + 0.5, \
                   pd_surv_df_atleast_i[outcome + '_T']) # set censored to max time
    pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] \
        = np.where(pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] >= 4.125 + 0.5, 4.125 + 0.5, \
                   pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)]) # set larger observed to max time
    outcome_T_atleast_i_cols.append(outcome + '_T_atleast' + str(4.125))

In [ ]:
pd_surv_df_atleast_i.head()

In [ ]:
outcome_T_atleast_i = [outcome + '_T_atleast4.125' for outcome in outcomes]
tsne_2comp_data = TSNE().fit_transform(pd_surv_df_atleast_i[outcome_T_atleast_i].values)

In [ ]:
tsne_1comp_data = TSNE(n_components=1).fit_transform(pd_surv_df_atleast_i[outcome_T_atleast_i].values)

In [ ]:
plt.scatter(tsne_2comp_data[:,0], tsne_2comp_data[:,1])
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('t-SNE on 5 outcome times with T=4.125')
plt.show()

In [ ]:
common_ts = [0., 0.125, 0.375, 0.625, 0.875, 1.125, 1.625, 2.125, 2.625, 3.125, 3.625, 4.125, 4.625]

In [ ]:
plt.rcParams.update({'font.size': 14})
for outcome in outcomes:
    T_early_idx = np.nonzero(np.where(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<1, 1, 0))[0]
    T_middle_idx = np.nonzero(np.where(np.logical_and(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']>=1, \
                                                      pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<2.5), \
                                       1, 0))[0]
    T_late_idx = np.nonzero(np.where(np.logical_and(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']>=2.5, \
                                                    pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<4.5), 1, 0))[0]
    T_at_end_idx = np.nonzero(np.where(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']==4.625, 1, 0))[0]
    assert len(T_early_idx) + len(T_middle_idx) + len(T_late_idx) + len(T_at_end_idx) == len(pd_surv_df_atleast_i)
    plt.scatter(tsne_2comp_data[T_early_idx,0], tsne_2comp_data[T_early_idx,1], color='green', label='0-0.875')
    plt.scatter(tsne_2comp_data[T_middle_idx,0], tsne_2comp_data[T_middle_idx,1], color='blue', label='1.125-2.125')
    plt.scatter(tsne_2comp_data[T_late_idx,0], tsne_2comp_data[T_late_idx,1], color='orange', label='2.625-4.125')
    plt.scatter(tsne_2comp_data[T_at_end_idx,0], tsne_2comp_data[T_at_end_idx,1], color='red', \
                label='at end (4.625)')
    plt.legend()
    plt.xlabel('Component 0')
    plt.ylabel('Component 1')
    plt.title('2-component t-SNE (colored by ' + outcome + ')')
    plt.savefig('tSNE_2comp_' + outcome + '.pdf')
    plt.show()

In [ ]:
plt.rcParams.update({'font.size': 14})
for outcome in outcomes:
    T_early_idx = np.nonzero(np.where(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<1, 1, 0))[0]
    T_middle_idx = np.nonzero(np.where(np.logical_and(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']>=1, \
                                                      pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<2.5), \
                                       1, 0))[0]
    T_late_idx = np.nonzero(np.where(np.logical_and(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']>=2.5, \
                                                    pd_surv_df_atleast_i[outcome + '_T_atleast4.125']<4.5), 1, 0))[0]
    T_at_end_idx = np.nonzero(np.where(pd_surv_df_atleast_i[outcome + '_T_atleast4.125']==4.625, 1, 0))[0]
    assert len(T_early_idx) + len(T_middle_idx) + len(T_late_idx) + len(T_at_end_idx) == len(pd_surv_df_atleast_i)
    plt.scatter(tsne_1comp_data[T_early_idx,0], np.random.sample(size=len(T_early_idx)), color='green', \
                label='0-0.875')
    plt.scatter(tsne_1comp_data[T_middle_idx,0], np.random.sample(size=len(T_middle_idx)), color='blue', \
                label='1.125-2.125')
    plt.scatter(tsne_1comp_data[T_late_idx,0], np.random.sample(size=len(T_late_idx)), color='orange', \
                label='2.625-4.125')
    plt.scatter(tsne_1comp_data[T_at_end_idx,0], np.random.sample(size=len(T_at_end_idx)), color='red', \
                label='at end (4.625)')
    plt.legend()
    plt.xlabel('Component 0')
    plt.ylabel('Random offset')
    plt.title('1-component t-SNE (colored by ' + outcome + ')')
    plt.savefig('tSNE_1comp_' + outcome + '.pdf')
    plt.show()

In [ ]:
pd_totals_df.EVENT_ID_DUR.value_counts()

In [ ]:
common_ts = [0., 0.125, 0.375, 0.625, 0.875, 1.125, 1.625, 2.125, 2.625, 3.125, 3.625, 4.125, 4.625]

In [ ]:
def make_pairwise_plot(outcome_i, outcome_j):
    pairwise_distribs = np.zeros((len(common_ts), len(common_ts)))
    for t_i in range(pairwise_distribs.shape[0]):
        for t_j in range(pairwise_distribs.shape[1]):
            pairwise_distribs[t_i, t_j] \
                = len(pd_surv_df_atleast_i.loc[np.logical_and(pd_surv_df_atleast_i[outcome_i + '_T_atleast4.125']==common_ts[t_i], \
                                                              pd_surv_df_atleast_i[outcome_j + '_T_atleast4.125']==common_ts[t_j])])
    pairwise_distribs_df = pd.DataFrame({outcome_i + '_T': common_ts})
    for t_j in range(pairwise_distribs.shape[1]):
        pairwise_distribs_df[common_ts[t_j]] = pairwise_distribs[:,t_j]
    pairwise_distribs_df = pairwise_distribs_df.set_index(outcome_i + '_T')
    fig, ax = plt.subplots(figsize=(10,8))
    ax = heatmap(pairwise_distribs_df, vmin=0, vmax=100, annot=True, ax=ax)
    ax.set_xlabel(outcome_j)
    ax.set_ylabel(outcome_i)
    plt.show()
             

In [ ]:
for i in range(len(outcomes)-1):
    for j in range(i+1, len(outcomes)):
        make_pairwise_plot(outcomes[i], outcomes[j])

### Plot inertia against number of clusters in kmeans
Inertia is the sum of squared distances from each sample to cluster center

In [ ]:
patnos_enrolled_at_least_i_years \
    = set(second_to_last_visits_df.loc[second_to_last_visits_df['EVENT_ID_DUR']>=4.125].PATNO.values.tolist())
pd_surv_df_atleast_i = pd_surv_df.loc[pd_surv_df['PATNO'].isin(patnos_enrolled_at_least_i_years)]
outcome_T_atleast_i_cols = []
for outcome in outcomes:
    pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] \
        = np.where(pd_surv_df_atleast_i[outcome + '_E']==0, 4.125 + 0.5, \
                   pd_surv_df_atleast_i[outcome + '_T']) # set censored to max time
    pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] \
        = np.where(pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)] >= 4.125 + 0.5, 4.125 + 0.5, \
                   pd_surv_df_atleast_i[outcome + '_T_atleast' + str(4.125)]) # set larger observed to max time
    outcome_T_atleast_i_cols.append(outcome + '_T_atleast' + str(4.125))

In [ ]:
all_patnos = pd_surv_df_atleast_i.PATNO.values
np.random.shuffle(all_patnos)
train_test_cutoff = int(.85*len(all_patnos))
train_patnos = set(all_patnos[:train_test_cutoff].tolist())
test_patnos = set(all_patnos[train_test_cutoff:].tolist())
train_df = pd_surv_df_atleast_i.loc[pd_surv_df_atleast_i['PATNO'].isin(train_patnos)]
test_df = pd_surv_df_atleast_i.loc[pd_surv_df_atleast_i['PATNO'].isin(test_patnos)]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
train_inertias = []
test_inertias = []
max_n_clust = 50
plt.rcParams.update({'font.size': 16})
for n_clust in range(1,max_n_clust):
    kmeans = KMeans(n_clusters=n_clust, random_state=7)
    kmeans.fit(train_df[outcome_T_atleast_i_cols])
    train_inertias.append(kmeans.inertia_/len(train_df))
    test_inertias.append(-1*kmeans.score(test_df[outcome_T_atleast_i_cols])/len(test_df))
plt.plot(range(1,max_n_clust), train_inertias, label='train')
plt.plot(range(1,max_n_clust), test_inertias, label='test')
plt.legend()
plt.xlabel('# clusters')
plt.ylabel('Inertia per sample')
plt.title('kmeans')
plt.tight_layout()
plt.savefig('kmeans_inertia_v_num_clust.pdf')
plt.show()

In [ ]:
len(train_df)

In [ ]:
len(test_df)